In [2]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.38GB/s]


In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


# **Importación de librerías y carga de archivos**

In [6]:
# Librerías básicas
import pandas as pd
import numpy as np

# Preprocesamiento y modelo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier

# Subir archivos desde tu PC (train.csv, test.csv, sample_submission.csv)
#from google.colab import files
#uploaded = files.upload()

# **Carga de datos y exploración básica**

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("submission_example.csv")

print("Dimensiones train:", train.shape)
print("Dimensiones test:", test.shape)
train.head()

Dimensiones train: (692500, 21)
Dimensiones test: (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# **Definición de la variable objetivo y tipos de variables**

In [8]:
# Columna objetivo
target_col = "RENDIMIENTO_GLOBAL"

y = train[target_col]
X = train.drop(columns=[target_col])

# Columnas numéricas.
num_cols = ["ID", "INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]

# El resto serán categóricas
cat_cols = [c for c in X.columns if c not in num_cols]

print("Numéricas:", num_cols)
print("Categóricas (ejemplo):", cat_cols[:10])


Numéricas: ['ID', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
Categóricas (ejemplo): ['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL']


# **División en entrenamiento y validación**

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("X_train:", X_train.shape, "X_valid:", X_valid.shape)


X_train: (554000, 20) X_valid: (138500, 20)


# **Preprocesamiento para variables numéricas y categoricas**

In [16]:
from sklearn.preprocessing import OrdinalEncoder

# Transformación para numéricas: solo imputación (árboles no necesitan escalado)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

# Transformación para categóricas: imputación + codificación ordinal
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinal", OrdinalEncoder(
        handle_unknown="use_encoded_value",
        unknown_value=-1
    ))
])

# ColumnTransformer: aplica numéricas y categóricas
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# **Definición del modelo de Regresión Logística**

In [17]:
# Modelo de regresión logística multinomial
model_hgb = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_depth=8,
    max_iter=200,   # antes 500
    l2_regularization=0.1,
    random_state=42
)

# **Construcción del pipeline completo (preprocesamiento + modelo)**

In [18]:
# Pipeline completo
pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model_hgb)
])

# **Entrenamiento, validación y métricas**

In [19]:
pipe.fit(X_train, y_train)
y_pred_valid = pipe.predict(X_valid)
print("Accuracy en valid:", accuracy_score(y_valid, y_pred_valid))
print(classification_report(y_valid, y_pred_valid))

Accuracy en valid: 0.431913357400722
              precision    recall  f1-score   support

        alto       0.54      0.63      0.58     35124
        bajo       0.46      0.57      0.51     34597
  medio-alto       0.33      0.26      0.29     34324
  medio-bajo       0.33      0.26      0.29     34455

    accuracy                           0.43    138500
   macro avg       0.41      0.43      0.42    138500
weighted avg       0.41      0.43      0.42    138500



# **Reentrenamiento con todo el conjunto de entrenamiento**

In [20]:
# Entrenamos el mismo pipeline con todos los datos de entrenamiento
pipe.fit(X, y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['ID', 'INDICADOR_1',
                                                   'INDICADOR_2', 'INDICADOR_3',
                                                   'INDICADOR_4']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))...
                                                   'E_VALORMATRICULAUNIVERSIDAD',
                                                   'E_HORASSEMANATRABAJA',
                                                   'F_ESTRATOVIVIENDA',
                                                   'F_TIENEINTERNET',
                                                   'F_EDUCACIONPADRE',
                                                   'F_TIENELAVADORA',
                                                   'F_TIENEAUTOMOVIL',
                                                   'E_PRIVADO_LIBERTAD',
                                                   'E_PAGOMATRICULAPROPIO',
                                                   'F_TIENECOMPUTADOR',
                                                   'F_TIENEINTERNET.1',
                                                   'F_EDUCACIONMADRE'])])),
                ('model',
                 HistGradientBoostingClassifier(l2_regularization=0.1,
                                                max_depth=8, max_iter=200,
                                                random_state=42))])

# **Predicciones para test y creación de my_submission.csv**

In [21]:
# Predicciones para el conjunto test
test_pred = pipe.predict(test)

# Revisamos columnas de sample_submission
print("Columnas de sample_submission:", sample_submission.columns.tolist())

# Asignamos las predicciones a la columna objetivo
# (asumimos que sample_submission tiene una columna RENDIMIENTO_GLOBAL)
sample_submission[target_col] = test_pred

# Guardar archivo de submission
sample_submission.to_csv("my_submission.csv", index=False)
print("Archivo my_submission.csv guardado.")
sample_submission.head()

Columnas de sample_submission: ['ID', 'RENDIMIENTO_GLOBAL']
Archivo my_submission.csv guardado.


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
